In [30]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import random
import string

In [31]:
# 1. Алгоритм шифра Цезаря
def caesar_cipher(text, shift):
    """Шифрует текст с помощью шифра Цезаря"""
    result = []
    for char in text.lower():
        if char in CHAR_TO_INDEX and char != 'none':
            idx = CHAR_TO_INDEX[char]
            new_idx = (idx - 1 + shift) % (len(INDEX_TO_CHAR) - 1)
            result.append(INDEX_TO_CHAR[new_idx + 1])
        else:
            result.append(' ')
    return ''.join(result)

In [32]:
# 2. Определяем словарь символов (только буквы + пробел)
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + list(CHARS)
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(INDEX_TO_CHAR)}

In [33]:
# 3. Генерация качественных тренировочных данных
def generate_training_data(num_samples=5000, max_length=20):
    """Генерирует качественные тренировочные данные"""
    phrases = []
    
    # Базовый словарь common words
    words = [
        'hello', 'world', 'ai', 'machine', 'learning', 'neural', 'network',
        'python', 'code', 'data', 'science', 'computer', 'vision', 'deep',
        'intelligence', 'algorithm', 'model', 'training', 'test', 'good',
        'the', 'and', 'for', 'with', 'this', 'that', 'from', 'your', 'have',
        'great', 'excellent', 'amazing', 'fantastic', 'wonderful', 'perfect'
    ]
    
    # Простые шаблоны фраз
    templates = [
        "{} {}", "{} {} {}", "{} {} {} {}", "the {} {}", "my {} {}",
        "this is {}", "i love {}", "we need {}", "let's try {}", "{} and {}"
    ]
    
    for _ in range(num_samples):
        # Выбираем случайный шаблон
        template = random.choice(templates)
        
        # Заполняем шаблон словами
        phrase = template.format(*random.sample(words, template.count('{}')))
        phrase = phrase[:max_length].lower().strip()
        
        # Убедимся, что фраза содержит только допустимые символы
        clean_phrase = ''.join([c for c in phrase if c in CHARS or c == ' '])
        if len(clean_phrase) >= 3:  # Минимальная длина
            phrases.append(clean_phrase)
    
    return phrases

In [34]:
# 4. Создаем датасет
class CaesarDataset(Dataset):
    def __init__(self, phrases, shift=3):
        self.phrases = phrases
        self.shift = shift
        self.max_len = 30
        
    def __len__(self):
        return len(self.phrases)
    
    def __getitem__(self, idx):
        original = self.phrases[idx].ljust(self.max_len, ' ')[:self.max_len]
        encrypted = caesar_cipher(original, self.shift)
        
        # Кодируем в числовой формат
        original_encoded = self.encode_text(original)
        encrypted_encoded = self.encode_text(encrypted)
        
        return encrypted_encoded, original_encoded
    
    def encode_text(self, text):
        encoded = torch.zeros(self.max_len, dtype=torch.long)
        for i, char in enumerate(text[:self.max_len]):
            encoded[i] = CHAR_TO_INDEX.get(char, 0)
        return encoded

In [35]:
# 5. Упрощенная и улучшенная модель
class SimpleCaesarNet(nn.Module):
    def __init__(self, vocab_size, embedding_dim=16, hidden_dim=64):
        super(SimpleCaesarNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True, num_layers=1)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.rnn(x)
        return self.fc(output)

In [36]:
# 6. Функция для вычисления accuracy
def calculate_accuracy(predictions, targets):
    """Вычисляет accuracy посимвольно"""
    _, predicted = torch.max(predictions, 2)
    correct = (predicted == targets).float()
    return correct.mean().item()

In [37]:
# 7. Генерация данных
print("Генерация тренировочных данных...")
phrases = generate_training_data(3000, 25)  # Меньше данных, но качественнее

print(f"Сгенерировано {len(phrases)} фраз")
print("Примеры фраз:", phrases[:5])

# Создаем датасет и DataLoader
dataset = CaesarDataset(phrases, shift=3)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Генерация тренировочных данных...
Сгенерировано 3000 фраз
Примеры фраз: ['this is have', 'i love data', 'for world computer neural', 'my the machine', 'the fantastic machine']


In [38]:
# 8. Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCaesarNet(len(INDEX_TO_CHAR)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Игнорируем padding
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [39]:
# 9. Обучение модели с мониторингом
num_epochs = 5
print("Начинаем обучение...")

for epoch in range(num_epochs):
    start_time = time.time()
    total_loss = 0
    total_acc = 0
    model.train()
    
    for encrypted, original in dataloader:
        encrypted = encrypted.to(device)
        original = original.to(device)
        
        optimizer.zero_grad()
        outputs = model(encrypted)
        
        # Вычисляем loss только для non-padding символов
        loss = criterion(outputs.view(-1, len(INDEX_TO_CHAR)), original.view(-1))
        acc = calculate_accuracy(outputs, original)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
        total_acc += acc
    
    scheduler.step()
    
    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Acc: {avg_acc:.2%}, Time: {time.time()-start_time:.2f}s')

Начинаем обучение...
Epoch 1/5, Loss: 1.6704, Acc: 62.22%, Time: 4.32s
Epoch 2/5, Loss: 0.3780, Acc: 94.63%, Time: 4.82s
Epoch 3/5, Loss: 0.0914, Acc: 99.48%, Time: 3.87s
Epoch 4/5, Loss: 0.0360, Acc: 99.91%, Time: 3.60s
Epoch 5/5, Loss: 0.0182, Acc: 100.00%, Time: 3.99s


In [46]:
# 10. Улучшенная функция дешифровки
def decrypt_caesar(text, model):
    """Дешифрует текст с помощью обученной модели"""
    model.eval()
    with torch.no_grad():
        # Кодируем входной текст
        encoded = torch.zeros(30, dtype=torch.long)
        clean_text = ''.join([c for c in text.lower() if c in CHARS or c == ' '])
        
        for i, char in enumerate(clean_text[:30]):
            encoded[i] = CHAR_TO_INDEX.get(char, 0)
        
        encoded = encoded.unsqueeze(0).to(device)
        outputs = model(encoded)
        
        # Получаем наиболее вероятные символы
        _, predicted = torch.max(outputs, 2)
        decrypted_chars = []
        
        for idx in predicted[0].cpu().numpy():
            if idx != 0:  # Пропускаем padding
                decrypted_chars.append(INDEX_TO_CHAR[idx])
        
        result = ''.join(decrypted_chars).strip()
        return result


In [48]:
# 11. Тестирование на простых примерах
print("\n" + "="*50)
print("ТЕСТИРОВАНИЕ МОДЕЛИ")
print("="*50)

# Тестируем только на одном сдвиге (том, на котором обучали)
test_phrases = [
    "hello world",
    "machine learning",
    "neural network",
    "deep learning",
    "artificial intelligence",
    "test phrase"
]

print("Тестируем на сдвиге 3:")
for phrase in test_phrases:
    encrypted = caesar_cipher(phrase, 3)
    decrypted = decrypt_caesar(encrypted, model)
    
    print(f"  Оригинал: {phrase}")
    print(f"  Зашифровано: {encrypted}")
    print(f"  Расшифровано: {decrypted}")
    print(f"  Правильно: {decrypted == phrase}")
    print()


ТЕСТИРОВАНИЕ МОДЕЛИ
Тестируем на сдвиге 3:
  Оригинал: hello world
  Зашифровано: zqjjdmpdwjg
  Расшифровано: mnggt ctygvfffffffffffffffffff
  Правильно: False

  Оригинал: machine learning
  Зашифровано: tiezolqmjqiwlolr
  Расшифровано: l gmmdn gn ydmdfffffffffffffff
  Правильно: False

  Оригинал: neural network
  Зашифровано: lqxwijmlqvpdwa
  Расшифровано: dnwn g dnactyudddfffffffffffff
  Правильно: False

  Оригинал: deep learning
  Зашифровано: gqqkmjqiwlolr
  Расшифровано: vnne gn ydmdffffffffffffffffff
  Правильно: False

  Оригинал: artificial intelligence
  Зашифровано: iwvoyoeoijmolvqjjorqleq
  Расшифровано: yamomgm g mdanggmfndgnddddfff
  Правильно: False

  Оригинал: test phrase
  Зашифровано: vqbvmkzwibq
  Расшифровано: aniamemy indddddffffffffffffff
  Правильно: False



In [42]:
# 12. Проверка на разных сдвигах (после дообучения)
def train_for_shift(shift, num_epochs=10):
    """Дообучаем модель для конкретного сдвига"""
    print(f"\nДообучение для сдвига {shift}...")
    
    dataset_shift = CaesarDataset(phrases, shift=shift)
    dataloader_shift = DataLoader(dataset_shift, batch_size=32, shuffle=True)
    
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        
        for encrypted, original in dataloader_shift:
            encrypted = encrypted.to(device)
            original = original.to(device)
            
            optimizer.zero_grad()
            outputs = model(encrypted)
            loss = criterion(outputs.view(-1, len(INDEX_TO_CHAR)), original.view(-1))
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"  Сдвиг {shift}, Epoch {epoch+1}, Loss: {total_loss/len(dataloader_shift):.4f}")

# Тестируем разные сдвиги последовательно
for test_shift in [1, 3, 5, 7]:
    train_for_shift(test_shift, num_epochs=5)
    
    print(f"\nРезультаты для сдвига {test_shift}:")
    for phrase in test_phrases[:3]:  # Только первые 3 для brevity
        encrypted = caesar_cipher(phrase, test_shift)
        decrypted = decrypt_caesar(encrypted, model)
        
        print(f"  {phrase} -> {decrypted} ({decrypted == phrase})")
    print()


Дообучение для сдвига 1...
  Сдвиг 1, Epoch 1, Loss: 1.4326
  Сдвиг 1, Epoch 2, Loss: 0.2468
  Сдвиг 1, Epoch 3, Loss: 0.1092
  Сдвиг 1, Epoch 4, Loss: 0.0650
  Сдвиг 1, Epoch 5, Loss: 0.0438

Результаты для сдвига 1:
  hello world -> hello worldggggggggggggggggggg (False)
  machine learning -> machine learninggggggggggggggg (False)
  neural network -> neural networkrggggggggggggggg (False)


Дообучение для сдвига 3...
  Сдвиг 3, Epoch 1, Loss: 0.3082
  Сдвиг 3, Epoch 2, Loss: 0.0432
  Сдвиг 3, Epoch 3, Loss: 0.0234
  Сдвиг 3, Epoch 4, Loss: 0.0162
  Сдвиг 3, Epoch 5, Loss: 0.0124

Результаты для сдвига 3:
  hello world -> hello worldngggggggggggggggggg (False)
  machine learning -> machine learninggggggggggggggg (False)
  neural network -> neural networkkggggggggggggggg (False)


Дообучение для сдвига 5...
  Сдвиг 5, Epoch 1, Loss: 1.7444
  Сдвиг 5, Epoch 2, Loss: 0.3046
  Сдвиг 5, Epoch 3, Loss: 0.1385
  Сдвиг 5, Epoch 4, Loss: 0.0855
  Сдвиг 5, Epoch 5, Loss: 0.0586

Результаты для